# 预处理assets，整理出后续要用的数据

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import json
import llms
os.environ["DEBUG"] = "1"
os.environ["SHOW_LLM_INPUT_MSG"] = "1"


In [2]:
# Preprocess the competition questions here
root_dir = os.getcwd()
question_data_path = root_dir + "/../../assets/question.json"
df1 = pd.read_excel(root_dir + "/../../assets/data_dictionary.xlsx", sheet_name="库表关系")
df2 = pd.read_excel(root_dir + "/../../assets/data_dictionary.xlsx", sheet_name="表字段信息")
file_path = root_dir + "/../../assets/all_tables_schema.txt"

In [3]:
# 假设 df1 的列名为 '库名英文', '库名中文', '表英文', '表中文', '表描述'
db_table = (
    df1.groupby("库名英文")
    .apply(
        lambda x: {
            "库名中文": x["库名中文"].iloc[0],
            "表": x[["表英文", "表中文", "表描述"]]
            .apply(lambda y: y.str.lower() if y.name == "表英文" else y)
            .to_dict("records"),
        }
    )
    .to_dict()
)
db_table = {k.lower(): v for k, v in db_table.items()}

os.makedirs(root_dir + "/assets", exist_ok=True)
with open(root_dir + "/assets/db_table.json", "w", encoding="utf-8") as json_file:
    json.dump(db_table, json_file, ensure_ascii=False, indent=2)

/var/folders/cz/76xt997d6kl_rwv3_7mx_nm40000gn/T/ipykernel_42959/1447885678.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [4]:
# 假设 df2 的列名为 'table_name', 'column_name', 'column_description', '注释'
table_column = (
    df2.groupby("table_name")
    .apply(
        lambda x: [
            {
                "column": row["column_name"],
                "desc": f"{row['column_description']}"
                + (f"；{row['注释']}" if pd.notna(row["注释"]) and row["注释"] else "").strip(),
            }
            for _, row in x.iterrows()
            if pd.notna(row["column_name"])
            and row["column_name"] not in ["JSID", "UpdateTime", "InsertTime", "ID", "XGRQ"]
        ]
    )
    .to_dict()
)
table_column = {k.lower(): v for k, v in table_column.items()}

# 打印结果以验证
with open(root_dir + "/assets/table_column.json", "w", encoding="utf-8") as json_file:
    json.dump(table_column, json_file, ensure_ascii=False, indent=2)

/var/folders/cz/76xt997d6kl_rwv3_7mx_nm40000gn/T/ipykernel_42959/3674784635.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [5]:
for db_name, db in db_table.items():
    for table in db["表"]:
        table_name = table["表英文"]
        print(f"{db_name}.{table_name}")
        column_list = []
        for column in table_column[table_name]:
            column_name = column["column"]
            column_simple_desc = str(column["desc"]).split("；")[0]
            column_list.append(f"{column_simple_desc}({column_name})")
        all_cols = ",".join(column_list)
        table["all_cols"] = all_cols
        cols_summary, token, ok = llms.llm_glm_4_plus.generate_response(
            system='''你善于对数据表的字段信息进行总结，把同类信息归类，比如"联系人电话、联系人传真"等总结为"联系方式如电话、传真等。
        输出一段文字，不换行。"''',
            messages=[
                {
                    "role": "user",
                    "content": f"下面是一个数据表的所有表字段，请帮我为这个数据表写一段介绍，把字段信息压缩进去：\n{all_cols}",
                }
            ],
            stream=False,
        )
        table["cols_summary"] = cols_summary
        if not ok:
            print(f"err: {db_name}.{table_name}")

astockbasicinfodb.lc_stockarchives
该数据表涵盖公司基本信息、联系方式、注册及办公地址、证券信息、高管及顾问详情、经营范围及特殊标识等。包括公司代码、国别、董事会秘书及其联系方式、证券事务代表及其联系方式、公司及董秘授权代表的电话、传真、邮箱，注册及办公地址及邮编，公司联系地址及邮编，公司网址、信息披露渠道，成立日期、注册地、法人代表、总经理、法律顾问、会计师事务所，证监会行业分类，主营兼营范围，A股、B股、H股及CDR证券简称及代码，公司简介、中文名称、营业执照号，地区代码，以及尚未盈利、特殊表决权、协议控制架构、红筹企业等特殊标识和所属区县信息。
astockbasicinfodb.lc_namechange
该数据表涵盖公司基本信息及股东大会决议情况，包括公司代码、信息发布日期及来源、股东大会决议公告日期、决议是否通过、公司全称及简称的更改日期，以及中英文全称和缩写。
astockbasicinfodb.lc_business
该数据表涵盖公司基本信息及业务范围，包括公司代码、信息发布日期及来源、股东大会决议公告日期、否决情况、主营及兼营范围、主要业务与产品名称、行业相关代码及类别、涉足行业以及简称变更原因等关键信息。
astockeventsdb.lc_warrant
该数据表涵盖公司事件相关信息，包括公司代码、信息发布日期及来源、公告类型与披露方式、事件内容描述及进展状态、事件主体及进程、行为方式、货币单位、主体与对象名称及编号、关联关系、协议签署及终止情况、担保原因及金额、担保企业及资产、反担保信息、借贷银行及期限、担保期限及解除详情、违规与逾期情况、事项编码等。
astockeventsdb.lc_credit
该数据表详细记录了公司借贷及相关事件的全面信息，包括公司代码、信息发布日期及来源、公告类型与披露方式、事件内容与行为描述、最新进展及主体信息、事件进程与行为方式、货币单位、主体与对象名称及编号、关联关系、协议签署及终止情况、借贷条件、借入借出方及担保方信息、抵押置押资产、借贷金额与还款情况、年利率及期限、借贷与担保起止日期等关键财务与事件数据。
astockeventsdb.lc_suitarbitration
该数据表涵盖公司事件相关信息，包括基础标识（ID、公司代码）、信息发布时间（首次及最新发布日期）、信

In [6]:
with open(root_dir + "/assets/db_table.json", "w", encoding="utf-8") as json_file:
    json.dump(db_table, json_file, ensure_ascii=False, indent=2)

In [7]:
db_info = []
for db_name, db in db_table.items():
    print(db_name)
    db_json = json.dumps(db, ensure_ascii=False)
    db_summary, token, ok = llms.llm_glm_4_plus.generate_response(
        system='''你善于对数据库的表信息进行总结，根据它包含的数据表和字段信息，描述这个数据库，如"本库名为xxx，记录了xxx；涵盖了xxx；方便用户xxx"。
输出一段文字，不换行。"''',
        messages=[
            {
                "role": "user",
                "content": f"下面是一个数据库的所有表和字段信息，请帮我为这个数据库写一段介绍，把表和字段信息压缩进去：\n{db}",
            }
        ],
        stream=False,
    )
    db_info.append(
        {
            "db_name": db_name,
            "db_desc": db_summary,
        }
    )

astockbasicinfodb
本库名为“上市公司基本资料”，记录了上市公司的核心信息及变更情况；涵盖了公司概况、名称更改状况、经营范围与行业变更三大板块。其中，“公司概况”表收录了上市公司的基本信息、联系方式、注册及办公地址、证券信息、高管及顾问详情、经营范围及特殊标识等；“公司名称更改状况”表记录了公司名称历次变更的详细信息；“公司经营范围与行业变更”表则详细展示了公司业务范围及行业类别的变动情况。方便用户全面了解上市公司的基本信息、历史变更及业务发展动态，为投资决策和行业研究提供有力支持。
astockeventsdb
本库名为“上市公司公告资讯/重大事项”，记录了上市公司公告中披露的各类重大事项信息；涵盖了公司担保明细、公司借贷明细、公司诉讼仲裁明细、重大事项委托理财、公司资产重组明细、公司重大经营合同明细、投资者关系活动及调研明细等多个方面；方便用户全面了解上市公司的财务状况、法律诉讼、资产重组、经营合同及投资者关系等重要信息，支持深入分析和决策。
astockfinancedb
本库名为上市公司财务指标/财务报表/融资与分红，记录了A股增发、A股配股、公司分红、资金投向说明、资产负债表_新会计准则、利润分配表_新会计准则、现金流量表_新会计准则、公司研发投入与产出、公司主营业务构成、公司经营情况述评、公司历年审计意见等数据，涵盖了证券发行全流程信息、配股相关信息、分红情况、募资及收购兼并相关信息、财务报表详细信息、财务信息、现金流量详细信息、研发投入及相关人员信息、经营与财务信息、经营状况及相关信息披露情况、审计相关信息等，方便用户全面了解上市公司的财务状况、经营成果、现金流量变动、研发投入、主营业务构成、经营情况、审计意见等信息。
astockindustrydb
本库名为“上市公司行业板块”，记录了上市公司在多种行业划分标准下的所属行业情况及其变更历史，涵盖了公司行业划分、行业变更、行业估值指标、行业财务指标以及公司所属概念等信息；方便用户进行公司数据回测、行业估值、财务分析及概念板块研究。具体包括公司行业划分表（lc_exgindustry）记录多级行业代码与名称，公司行业变更表（lc_exgindchange）追踪行业变化，行业估值指标表（lc_industryvaluation）提供市盈率、市净率等财务指标，行业财务指标表（lc_indfi

In [8]:
with open(root_dir + "/assets/db_info.json", "w", encoding="utf-8") as json_file:
    json.dump(db_info, json_file, ensure_ascii=False, indent=2)